In [1]:
from pygame.locals import *
import pygame
from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *
import math
from OpenGL.GL.shaders import *

from geo_math import *
   

def cauculate_center(verticies):
    center = [0, 0, 0]
    for vertex in verticies:
        center = [center[i] + vertex[i] for i in range(3)]
    center = [center[i] / len(verticies) for i in range(3)]
    return center
 
def calculate_surface_normal(triangle):

    u = [i-j for i,j in zip(triangle[1], triangle[0])]
    v = [i-j for i,j in zip(triangle[2], triangle[0])]

    return [
        u[1]*v[2] - u[2]*v[1],
        u[2]*v[0] - u[0]*v[2],
        u[0]*v[1] - u[1]*v[0]
    ]
    
def rotate2(subject, pivot, angle):
        subject_x, subject_y = subject
        pivot_x, pivot_y = pivot
        a = angle * math.pi / 180
        dx = subject_x - pivot_x
        dy = subject_y - pivot_y
        r = math.sqrt(dx * dx + dy * dy)
        theta = math.atan2(dx, pivot_y - subject_y) - math.pi / 2
        return [
            pivot_x + r * math.cos(theta + a),
            pivot_y + r * math.sin(theta + a),
        ]
    
def rotate(suject, pivot, angle):
    angle_x, angle_y, angle_z = angle
    pivot_x, pivot_y, pivot_z = pivot
    subject_x, subject_y, subject_z = suject

    subject_x, subject_z = rotate2([subject_x, subject_z], [pivot_x, pivot_z], angle_x)
    subject_y, subject_z = rotate2([subject_y, subject_z], [pivot_y, pivot_z], angle_y)
    subject_x, subject_y = rotate2([subject_x, subject_y], [pivot_x, pivot_y], angle_z)

    return [subject_x, subject_y, subject_z]


plane1 = [
    [-1, -1],
    [-1, 1],
    [1, 1],
    [1, -1],
]

plane2 = [
    [-1, -1],
    [-1, 1],
    [0, 2],
    [1, 1],
    [1, -1],
    [0, -2],
]

no_rotation = [0, 0, 0]
rotation = [0, 0, 20]

no_scale = [1, 1, 1]
sacale = [0.5,2,0.5]

base = [1,1]
half = [0.5,0.5]
small = [0.4,0.4]

parameters = [
    [plane1, 0,   base,  no_rotation],
    [plane2, 1.5, half,  no_rotation],
    [plane2, 1.5, half,  no_rotation],
    [plane1, 1.5, small, no_rotation],
    [plane1, 1,   small, rotation],
    [plane1, 1,   small, rotation],
]

pygame.init()
pygame.display.set_caption('PyGame OpenGL')
pygame.display.set_icon(pygame.image.load('icon.png'))
screen = (800, 600)
pygame.display.set_mode(screen, DOUBLEBUF|OPENGL)

glEnable(GL_DEPTH_TEST)

glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA)
glEnable(GL_BLEND)
# glEnable(GL_LIGHTING)
# glEnable(GL_LIGHT0)
# glEnable(GL_COLOR_MATERIAL)

glMatrixMode(GL_PROJECTION)
gluPerspective(45, (screen[0]/screen[1]), 0.1, 50.0)

glMatrixMode(GL_MODELVIEW)  

rot_x, rot_y, zoom = 30, 45, -10
glClearColor(0, 0.4, 0.7, 1)

clock = pygame.time.Clock()
busy = True
mv_mat = (GLdouble * 16)()
p_mat  = (GLdouble * 16)()
v_rect = (GLint * 4)() 
selected_id = -1
while busy:
    try:
        mouse_down = pygame.mouse.get_pressed()
        keyboard_down = pygame.key.get_pressed()
        left_click = mouse_down[0] == 1
        left_ctrl = keyboard_down[pygame.K_LCTRL]
    
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                busy = False
            elif event.type == pygame.MOUSEMOTION:
                if left_click and left_ctrl:
                    rot_x = (rot_x + event.rel[1]) % 360
                    if rot_x > 90 and rot_x < 270:
                        rot_y = (rot_y - event.rel[0]) % 360
                    else:
                        rot_y = (rot_y + event.rel[0]) % 360
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 4:
                    zoom += 2
                if event.button == 5:
                    zoom -= 2
                
        glClear(GL_COLOR_BUFFER_BIT|GL_DEPTH_BUFFER_BIT)
        glPushMatrix()
        # glLight(GL_LIGHT0, GL_POSITION,  (0, 1, 0, 1))
        # glLightfv(GL_LIGHT0, GL_AMBIENT,  (1, 1, 1, 0))
        # glColorMaterial(GL_FRONT_AND_BACK, GL_AMBIENT_AND_DIFFUSE)
        glTranslatef(0.0,0.0, zoom)
        glRotatef(rot_x, 1, 0, 0)  
        glRotatef(rot_y, 0, 1, 0)  
        glGetDoublev(GL_MODELVIEW_MATRIX, mv_mat)
        glGetDoublev(GL_PROJECTION_MATRIX, p_mat)
        glGetIntegerv(GL_VIEWPORT, v_rect)
        mouse_pos = pygame.mouse.get_pos()
        mouse_pos = mouse_pos[0], v_rect[3] - mouse_pos[1]

        temp_val = [GLdouble() for _ in range(3)]
        OpenGL.raw.GLU.gluUnProject(*mouse_pos, 0, mv_mat, p_mat, v_rect, *temp_val)
        mouse_near = [v.value for v in temp_val]    
        OpenGL.raw.GLU.gluUnProject(*mouse_pos, 1, mv_mat, p_mat, v_rect, *temp_val)
        mouse_far = [v.value for v in temp_val]
        verticies = []
        edegs = []
        faces = []
        overlay_edegs = []
        overlay_faces = []
        index = 0
        previus_index = 0
        previus_plane = None
        z = 0
        previus_center = cauculate_center([[vertex[0], 0, vertex[1]] for vertex in parameters[0][0]])
        rotation = [0, 0, 0]
        rotation_index = [0, 0, 0]
        for section_id in range(len(parameters)):
            plane, z_relative, scale, rotation_relative = parameters[section_id]

            z += z_relative

            rotation_index = [rotation_index[i] + rotation_relative[i] for i in range(3)]
            rotation = [rotation[i] + rotation_index[i] for i in range(3)]

            plane = [[j * s for j, s in zip(i, scale)] for i in plane]
            
            cur_verticies = [rotate([v[0], z, v[1]], previus_center, rotation) for v in plane]
            verticies += cur_verticies

            center = cauculate_center(cur_verticies)

            boundry_edegs = [[index+v, index+((v-1) % len(plane))] for v in range(len(plane))]
            edegs += boundry_edegs
            overlay_edegs += [boundry_edegs]

            if(index > 0):

                length = len(plane)
                previus_length = len(previus_plane)
                
                if(length > previus_length):
                    bigger_length = length
                    smaller_length = previus_length
                else:
                    bigger_length = previus_length
                    smaller_length = length

            
                if(length == previus_length):
                    for j in range(bigger_length):
                        a = index + j
                        b = previus_index + j
                        pa = index + (j-1) % length
                        pb = previus_index + (j-1) % previus_length

                        edegs += [[a, b]]
                        edegs += [[pa, b]]

                        faces += [[pa,b , a]]
                        faces += [[pa, pb, b]]

                elif(length > previus_length):
                    for j in range(bigger_length):
                        norm_b = j / length * previus_length

                        a  = index + math.floor(j)
                        b  = math.floor(previus_index + norm_b)
                        c = previus_index + math.ceil(norm_b) % smaller_length
                        pa = math.floor(index + ((j-1) % length))
                        
                        edegs += [[a, b]]
                        edegs += [[a, c]]

                        faces += [[a, b, c]]
                        faces += [[a, pa, b]]

                else:
                    for j in range(bigger_length):
                        norm_a = j / previus_length * length

                        a  = index + math.floor(norm_a)
                        b  = math.floor(previus_index + j)
                        c = index + math.ceil(norm_a) % smaller_length
                        pb = math.floor(previus_index + ((j-1) % previus_length))

                        edegs += [[a, b]]
                        edegs += [[c, b]]

                        faces += [[a, b, c]]
                        faces += [[a, pb, b]]
            
            previus_index = index
            previus_plane = plane
            previus_center = center

            verticies += [center]
            center_index = len(verticies)-1
            current_faces = []
            for i in range(len(plane)):
                current_faces += [[center_index, index+i, index+((i-1) % len(plane))]]
            overlay_faces += [current_faces]
            if section_id == 0:
                for i in range(len(plane)):
                    edegs += [[center_index, index+i]]
                    faces += [[center_index, index+i, index+((i-1) % len(plane))]]
            elif section_id == len(parameters)-1:
                
                for i in range(len(plane)):
                    edegs += [[center_index, index+i]]
                    faces += [[ index+((i-1) % len(plane)), index+i, center_index]]
            index += len(plane)+1

        center = cauculate_center(verticies)

        for vertex in verticies:
            for coordinate in range(3):
                vertex[coordinate] = vertex[coordinate] - center[coordinate]
        glEnable(GL_POLYGON_OFFSET_FILL)
        glPolygonOffset(1.0, 1.0)


        glBegin(GL_TRIANGLES)
        glColor3fv((0.4,0.4,0.4))    
        for face in faces:
            # glNormal3fv(calculate_surface_normal([verticies[vertex] for vertex in face]))
            for vertex in face:
                glVertex3fv(verticies[vertex])
        glEnd()

        glBegin(GL_LINES)
        glColor3fv((1,1,1))
        for edeg in edegs:
            for vertex in edeg:
                glVertex3fv(verticies[vertex])
        glEnd()

        glClear(GL_DEPTH_BUFFER_BIT)

        smallest_distance = math.inf
        hover_id = -1
        for i in range(len(overlay_faces)):
            for face in overlay_faces[i]:
                distance = isectTrianlge(mouse_near, mouse_far, verticies[face[0]], verticies[face[1]], verticies[face[2]])
                if(distance):
                    if(distance < smallest_distance):
                        smallest_distance = distance
                        hover_id = i
                    break


        def draw_overlay(id, color):
            glBegin(GL_TRIANGLES)
            glColor4fv((*color, 0.2)) 
            for segment in overlay_faces[id]:
                for vertex in segment:
                    glVertex3fv(verticies[vertex])
            glEnd()
            
            glBegin(GL_LINES)
            glColor3fv(color)
            for edeg in overlay_edegs[id]:
                for vertex in edeg:
                    glVertex3fv(verticies[vertex])
            glEnd()

        if hover_id != -1:
            if(left_click and not left_ctrl):
                selected_id = hover_id
        else:
            if(left_click and not left_ctrl):
                selected_id = -1

        if(selected_id != -1):
            draw_overlay(selected_id, (1,1,0))
            glClear(GL_DEPTH_BUFFER_BIT)
        
        if(hover_id != -1 and hover_id != selected_id):
            draw_overlay(hover_id, (0,1,1))

        
        glPopMatrix()
        
        pygame.display.flip()
        clock.tick(100)
    except Exception as e:
        pygame.quit()
        raise e
        
pygame.quit()

pygame 2.1.0 (SDL 2.0.16, Python 3.9.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
